In [8]:
import pandas as pd
import numpy as np
import json

In [ ]:
data = pd.read_csv('updated_full_patients.csv')

In [ ]:
# find shape of data
data.shape

(167034, 12)

In [ ]:
# find the columns and dtypes
data.columns

Index(['patient_id', 'patient_uid', 'patient', 'title', 'age', 'age_units',
       'gender', 'similar_patients', 'diseases_from_patient',
       'diseases_from_title', 'diseases_from_both', 'disease'],
      dtype='object')

In [ ]:
# find how many rows are empty for each of the columns
data.isnull().sum()

,0
patient_id,0
patient_uid,0
patient,0
title,0
age,0
age_units,0
gender,0
similar_patients,0
diseases_from_patient,39464
diseases_from_title,81747


In [ ]:
# find how many rows has disease column as empty array
#data[data['disease'].apply(lambda x: x == [])]
#data[data['disease'].isnull()]

,patient_id,patient_uid,patient,title,age,age_units,gender,similar_patients,diseases_from_patient,diseases_from_title,diseases_from_both,disease


In [ ]:
# find rows where diseases_from_patient , disease_from_title, disease_from_both and disease are blank
#disease will be empty array
data[(data['diseases_from_patient'].isnull()) & (data['diseases_from_title'].isnull()) & (data['diseases_from_both'].isnull()) & (data['disease'].isnull())]
#store in sepearte df and print number of rows
blank_disease = data[(data['diseases_from_patient'].isnull()) & (data['diseases_from_title'].isnull()) & (data['diseases_from_both'].isnull()) & (data['disease'].isnull())]

In [ ]:

data['disease'].apply(lambda x: x in [None, '', '[]',[],'NULL',np.nan]).sum()

25491

In [ ]:
# Function to clean the text by removing special characters and stripping whitespace
import re
def clean_text(value):
    if isinstance(value, str):
        return re.sub(r'[^a-zA-Z0-9\s]', '', value).strip()
    return value

# Apply the cleaning function to the 'disease' column
data['disease_cleaned'] = data['disease'].apply(clean_text)

# Replace empty strings with NaN
data['disease_cleaned'] = data['disease_cleaned'].replace('', np.nan)
# Replace fully empty or blank strings with NaN
data['disease_cleaned'].replace(['', ' ', 'nan', 'NaN', 'NULL', 'null'], pd.NA, inplace=True)

# Count the number of rows where 'disease_cleaned' is NaN
empty_disease_cleaned_count = data['disease_cleaned'].isna().sum()

# Get the unique values in 'disease_cleaned' column
unique_disease_cleaned_values = data['disease_cleaned'].dropna().unique()

# Output the results
print(f"Row count with empty 'disease_cleaned': {empty_disease_cleaned_count}")

Row count with empty 'disease_cleaned': 25501


<ipython-input-7-d8fb1fc14d87>:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['disease_cleaned'].replace(['', ' ', 'nan', 'NaN', 'NULL', 'null'], pd.NA, inplace=True)


In [ ]:
# Define columns to check for emptiness
columns_to_check = ['diseases_from_patient', 'diseases_from_title', 'diseases_from_both']

# Count rows where 'disease_cleaned' and all related columns are empty or NaN
rows_with_all_empty = data.loc[
    data['disease_cleaned'].isna() &
    data[columns_to_check].apply(lambda x: x.isna() | x.eq('')).all(axis=1)
]

rows_with_all_empty.to_csv('TEST 2.csv',index = False)
# Get the count of such rows
count_all_empty_rows = rows_with_all_empty.shape[0]

# Create a DataFrame with those rows removed
cleaned_data = data.drop(rows_with_all_empty.index)

# Output the results
print(f"Count of rows where 'disease_cleaned' and all related columns are empty: {count_all_empty_rows}")

Count of rows where 'disease_cleaned' and all related columns are empty: 25501


In [ ]:
cleaned_data.to_csv('Cleaned.csv',index=False)

In [ ]:
# Issue in excel is because of this patient overflowingtext


cleaned_data[cleaned_data['patient_id']==19714]

,patient_id,patient_uid,patient,title,age,age_units,gender,similar_patients,diseases_from_patient,diseases_from_title,diseases_from_both,disease,disease_cleaned
19714,19714,7237229-1,"Case2: A 49 year old immunocompetent male, pre...",Abstracts from the International Science Sympo...,49,year,M,"{'7237229-2': 2, '7237229-3': 2, '7237229-4': 2}","klebsiella pneumonia, autoimmune disease, ence...",NaN,NaN,"klebsiella pneumonia, autoimmune disease, ence...",klebsiella pneumonia autoimmune disease enceph...


Deleted rows where all 4 disease columns were blank. that is also stored in test csv. Below now again analysing the remaining data.

In [ ]:
# find how many rows are not empty for each of the columns
cleaned_data.isnull().sum()

,0
patient_id,0
patient_uid,0
patient,0
title,0
age,0
age_units,0
gender,0
similar_patients,0
diseases_from_patient,13963
diseases_from_title,56246


In [ ]:
def is_effectively_empty(value):
    if pd.isna(value) or value in ['', ' ', 'nan', 'NaN', 'NULL', 'null']:
        return True
    if isinstance(value, str) and value.strip() == '':
        return True
    return False

# Apply the check to each column and count non-empty rows
non_empty_counts = cleaned_data.apply(lambda col: (col.apply(is_effectively_empty)).sum())

# Output the non-empty counts for each column
print("Empty row counts for each column:")
print(non_empty_counts)

Empty row counts for each column:
patient_id                   0
patient_uid                  0
patient                      0
title                        0
age                          0
age_units                    0
gender                       0
similar_patients             0
diseases_from_patient    13963
diseases_from_title      56246
diseases_from_both       89130
disease                      0
disease_cleaned              0
dtype: int64


In [ ]:
cleaned_data.shape

(141533, 13)

In [ ]:
# Select a random 5% sample of the data
sample_data = cleaned_data.sample(frac=0.05, random_state=42)

# Perform quality checks on the sample data
# 1. Check for any missing values in critical columns
missing_values_check = sample_data.isna().sum()

# 2. Check for duplicate records
duplicate_records_check = sample_data.duplicated().sum()

# 3. Validate data types (for example, checking if patient_uid is numeric)
data_type_check = sample_data.dtypes

# 4. Check for outliers in numeric columns (e.g., using summary statistics)
summary_statistics = sample_data.describe()

# Output the quality check results
print("Missing values check:\n", missing_values_check)
print("\nDuplicate records check:", duplicate_records_check)
print("\nData type check:\n", data_type_check)
print("\nSummary statistics:\n", summary_statistics)

# Save the sample data to a CSV file (optional)
sample_data.to_csv('sample_data_for_quality_check.csv', index=False)
print("Sample data saved to 'sample_data_for_quality_check.csv'")

Missing values check:
 patient_id                  0
patient_uid                 0
patient                     0
title                       0
age                         0
age_units                   0
gender                      0
similar_patients            0
diseases_from_patient     722
diseases_from_title      2772
diseases_from_both       4417
disease                     0
disease_cleaned             0
dtype: int64

Duplicate records check: 0

Data type check:
 patient_id                int64
patient_uid              object
patient                  object
title                    object
age                       int64
age_units                object
gender                   object
similar_patients         object
diseases_from_patient    object
diseases_from_title      object
diseases_from_both       object
disease                  object
disease_cleaned          object
dtype: object

Summary statistics:
           patient_id          age
count    7077.000000  7077.000000
mean   

Now some rows have multiple diseases, do we keep them all for training or do we keep only one disease per row ?

In [ ]:
# Define a function to handle comma-separated diseases
def has_multiple_diseases(value):
    if isinstance(value, str):
        # Split by commas, trim whitespace, and count non-empty diseases
        diseases = [d.strip() for d in value.split(',') if d.strip()]
        return len(diseases) > 1
    return False

# Apply the function to the 'disease_from_title' column and count rows with multiple diseases
multiple_diseases_count = data['diseases_from_title'].apply(has_multiple_diseases).sum()

# Output the result
print(f"Number of rows with multiple comma-separated diseases in 'disease_from_title' column: {multiple_diseases_count}")

Number of rows with multiple comma-separated diseases in 'disease_from_title' column: 37806


In [ ]:
#count rows where column diseases_from_title is not null
cleaned_data['diseases_from_title'].notnull().sum()

85287

Total Rows: 141533  
Total rows where disease_from_title is not null = 85287  
Total rows where disease_from_title has multiple disease = 37806  


Now that I have stripped spl chars from disease column, Checking the rows where all other disease column are empty and we need data from the disease column.  

In [ ]:
def is_empty_or_blank(value):
    return pd.isna(value) or (isinstance(value, str) and value.strip() == '')

# Count rows where all three columns are empty, null, or blank
all_empty_rows_count = cleaned_data.apply(
    lambda row: all(is_empty_or_blank(row[col]) for col in ['diseases_from_title', 'diseases_from_patient', 'diseases_from_both']),
    axis=1
).sum()

# Output the result
print(f"Number of rows where 'disease_from_title', 'disease_from_patient', and 'disease_from_both' are all empty or null: {all_empty_rows_count}")

Number of rows where 'disease_from_title', 'disease_from_patient', and 'disease_from_both' are all empty or null: 3670


In [ ]:
# Define a function to check if a value is empty, null, or blank
def is_empty_or_blank(value):
    return pd.isna(value) or (isinstance(value, str) and value.strip() == '')

# Count rows where all specified columns are empty, but 'disease' is not empty
filtered_rows_count = cleaned_data.apply(
    lambda row: all(is_empty_or_blank(row[col]) for col in ['diseases_from_title', 'diseases_from_patient', 'diseases_from_both']) and
                not is_empty_or_blank(row['disease_cleaned']),
    axis=1
).sum()

# Output the result
print(f"Number of rows where 'disease_from_title', 'disease_from_patient', and 'disease_from_both' are all empty, but 'disease' is not empty: {filtered_rows_count}")

Number of rows where 'disease_from_title', 'disease_from_patient', and 'disease_from_both' are all empty, but 'disease' is not empty: 3670


In [ ]:
def is_empty_or_blank(value):
    return pd.isna(value) or (isinstance(value, str) and value.strip() == '')

# Filter rows where the specified columns are empty but 'disease' is not empty
filtered_rows_df = cleaned_data[
    cleaned_data.apply(
        lambda row: all(is_empty_or_blank(row[col]) for col in ['diseases_from_title', 'diseases_from_patient', 'diseases_from_both']) and
                    not is_empty_or_blank(row['disease']),
        axis=1
    )
]

# Save the filtered rows to a CSV file
filtered_rows_file = 'filtered_rows_with_disease.csv'
filtered_rows_df.to_csv(filtered_rows_file, index=False)

print(f"Filtered rows saved to '{filtered_rows_file}'")

Filtered rows saved to 'filtered_rows_with_disease.csv'


out of 3670 records, 180 were roughly incorrect/in accurate. ~5% for the spacy column.

In [ ]:
# Define a function to handle comma-separated diseases
def has_multiple_diseases(value):
    if isinstance(value, str):
        # Split by commas, trim whitespace, and count non-empty diseases
        diseases = [d.strip() for d in value.split(',') if d.strip()]
        return len(diseases) > 1
    return False

# Apply the function to the 'disease_from_title' column and count rows with multiple diseases
multiple_diseases_count = data['diseases_from_patient'].apply(has_multiple_diseases).sum()

# Output the result
print(f"Number of rows with multiple comma-separated diseases in 'disease_from_patient' column: {multiple_diseases_count}")

Number of rows with multiple comma-separated diseases in 'disease_from_patient' column: 93559


In [ ]:
# removing the rows with only spacy output diseases.
# Filter rows where at least one of the three columns is not empty, null, or blank
filtered_data = cleaned_data[
    ~cleaned_data.apply(lambda row: all(is_empty_or_blank(row[col]) for col in ['diseases_from_title', 'diseases_from_patient', 'diseases_from_both']), axis=1)
]

# Now, filtered_data contains only rows where at least one column has a value
print(f"Shape of filtered data: {filtered_data.shape}")


Shape of filtered data: (137863, 13)


In [ ]:
# save filtered data to csv
filtered_data.to_csv('filtered_data.csv',index=False)

Now below I am going to pick a sample of 7k rows which is 5% of data to check quality.

In [ ]:

# Sample 6894 rows from the cleaned data 5% of 137863
sampled_data = cleaned_data.sample(n=6894, random_state=42)

# Split the sampled data into two DataFrames of 3500 records each
df_part1 = sampled_data.iloc[:3500]
df_part2 = sampled_data.iloc[3500:]

# Save each DataFrame to a separate CSV file
df_part1.to_csv('sampled_data_part1.csv', index=False)
df_part2.to_csv('sampled_data_part2.csv', index=False)

print("Sampled data saved to 'sampled_data_part1.csv' and 'sampled_data_part2.csv'")

Sampled data saved to 'sampled_data_part1.csv' and 'sampled_data_part2.csv'


In [ ]:
# Sample 6894 records from cleaned_data 5% of 137863
sampled_data = filtered_data.sample(n=6894, random_state=42)

# Determine split sizes
split1 = 2300
split2 = 4600  # This ensures the remaining records go into the third part

# Split the sampled data into three DataFrames
df_part1 = sampled_data.iloc[:split1]
df_part2 = sampled_data.iloc[split1:split2]
df_part3 = sampled_data.iloc[split2:]

# Save each DataFrame to a separate CSV file
df_part1.to_csv('sampled_part1.csv', index=False)
df_part2.to_csv('sampled_part2.csv', index=False)
df_part3.to_csv('sampled_part3.csv', index=False)

print("Sampled data saved to 'sampled_part1.csv', 'sampled_part2.csv', and 'sampled_part3.csv'")


Sampled data saved to 'sampled_part1.csv', 'sampled_part2.csv', and 'sampled_part3.csv'


overall skimming of csv's show good quality. Decision to pick from the column and how many or which disease to choose is a challenge.

In [ ]:
filtered_data = pd.read_csv('filtered_data.csv')

In [ ]:
filtered_data.head(3)

,patient_id,patient_uid,patient,title,age,age_units,gender,similar_patients,diseases_from_patient,diseases_from_title,diseases_from_both,disease,disease_cleaned
0,0,7665777-1,This 60-year-old male was hospitalized due to ...,Early Physical Therapist Interventions for Pat...,60,year,M,"{'7665777-2': 2, '7665777-3': 2, '7665777-4': ...","respiratory failure, covid-19",covid-19,covid-19,covid-19,covid19
1,1,7665777-2,A 39-year-old man was hospitalized due to an i...,Early Physical Therapist Interventions for Pat...,39,year,M,"{'7665777-1': 2, '7665777-3': 2, '7665777-4': ...",anxiety,covid-19,NaN,covid-19,covid19
2,2,7665777-3,One week after a positive COVID-19 result this...,Early Physical Therapist Interventions for Pat...,57,year,M,"{'7665777-1': 2, '7665777-2': 2, '7665777-4': ...",covid-19,covid-19,covid-19,covid-19,covid19


In [ ]:
# filter rows to a df where diseases_from_title is not null
df_title = filtered_data[filtered_data['diseases_from_title'].notnull()]

In [ ]:
df_title.shape

(85287, 13)

In [ ]:
# save df_title to csv
df_title.to_csv('df_title.csv',index=False)

In [ ]:
# from df_title create a sample dataframe of 5%
df_title_sample = df_title.sample(frac=0.05, random_state=42)

In [ ]:
# store df_title_sample in to csv after splitting into 2 parts
df_title_sample.iloc[:3500].to_csv('df_title_sample_part1.csv',index=False)
df_title_sample.iloc[3500:].to_csv('df_title_sample_part2.csv',index=False)

In [3]:
dataset = pd.read_csv('df_title.csv')

In [4]:
dataset.head(2)

,patient_id,patient_uid,patient,title,age,age_units,gender,similar_patients,diseases_from_patient,diseases_from_title,diseases_from_both,disease,disease_cleaned
0,0,7665777-1,This 60-year-old male was hospitalized due to ...,Early Physical Therapist Interventions for Pat...,60,year,M,"{'7665777-2': 2, '7665777-3': 2, '7665777-4': ...","respiratory failure, covid-19",covid-19,covid-19,covid-19,covid19
1,1,7665777-2,A 39-year-old man was hospitalized due to an i...,Early Physical Therapist Interventions for Pat...,39,year,M,"{'7665777-1': 2, '7665777-3': 2, '7665777-4': ...",anxiety,covid-19,NaN,covid-19,covid19


In [5]:
# filter dataset to have columns patient and disease_from_title
dataset = dataset[['patient','diseases_from_title']]

In [7]:
dataset.head(5)

,patient,diseases_from_title
0,This 60-year-old male was hospitalized due to ...,covid-19
1,A 39-year-old man was hospitalized due to an i...,covid-19
2,One week after a positive COVID-19 result this...,covid-19
3,This 69-year-old male was admitted to the ICU ...,covid-19
4,This 57-year-old male was admitted to the ICU ...,covid-19


In [9]:
jsonl_file = "medical_prompts.jsonl"

with open(jsonl_file, "w") as f:
    for _, row in dataset.iterrows():
        prompt = {
            "prompt": f"You are a medical patient summary generator given diseases {row['diseases_from_title']}, generate a patient summary.",
            "response": row["patient"]
        }
        f.write(json.dumps(prompt) + "\n")

print(f"JSONL file '{jsonl_file}' created successfully.")

JSONL file 'medical_prompts.jsonl' created successfully.
